# 项目分析

逐级生成分析，使用不同模板。

粗描述生成（自上而下）：
1. 首先读取每个文件，针对每个文件生成粗略描述（限制输入文本长度以免避免模型超限）
2. 根据每个文件，生成文件命名空间（父文件夹）描述（大部分引用的为命名空间）
3. 之后针对每个文件中的类生成描述，利用文件引用将其他文件的粗描述引入作为信息
4. 针对每个文件中的方法生成描述，利用类的描述信息

精细描述生成（自下而上）：
1. 利用一个类中的其他方法的描述，以及类的描述，生成当前方法描述
2. 利用类中所有方法的精细描述，生成类的描述
3. 利用一个文件中所有类的描述，生成文件的描述
4. 结合引用文件描述，重新生成当前文件描述

In [2]:
import time
import os
import pickle
from typing import *

In [3]:
import torch

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
from tinydb import TinyDB,Query

In [6]:
os.environ["RWKV_CUDA_ON"] = 'cuda:1' # if '1' then use CUDA kernel for seq mode (much faster)

In [7]:
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'

In [8]:
device = 'cuda:1' if torch.cuda.is_available() else '0'

In [9]:
def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read()
    return detect(rawdata)['encoding']

## 1. 读取项目测试数据

In [10]:
import json

In [11]:
from chardet import detect

## 2. 读取模型参数测试

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("../data/deepseek-coder-6.7b-instruct-AQLM/", 
                                             # device_map = 'auto',
                                             trust_remote_code=True, 
                                             torch_dtype=torch.bfloat16).cuda()

`low_cpu_mem_usage` was None, now default to True since model is quantized.


## 3. 项目分析提示词模板

In [14]:
import tinydb

#### 自上而下-类结构描述

In [15]:
file_db_lst = tinydb.TinyDB('../data/results/amc-rough_csfile_db-dscoderv2lite.json').all()
namespace_db_lst = tinydb.TinyDB('../data/results/amc-rough_namespace_db-dscoderv2lite.json').all()
class_db_lst = tinydb.TinyDB('../data/results/amc-rough_class_db-dscoderv2lite.json').all()
method_db_lst = tinydb.TinyDB('../data/results/amc-rough_method_db-dscoderv2lite.json').all()

In [16]:
file_db_map = {n['filepath']:n['desc'] for n in file_db_lst}

namespace_db_map = {n['namespace']:n['desc'] for n in namespace_db_lst}

class_db_map = {}
for cnode in class_db_lst:
    if not cnode['filepath'] in class_db_map:
        class_db_map[cnode['filepath']] = {}
    class_db_map[cnode['filepath']][cnode['classname']] = cnode['desc']

project_meta = json.load(open('../data/processed/amc-by-tree-sitter.json','r'))

method_db_map = {}
for node in method_db_lst:
    fpath = node['filepath']
    class_name = node['classname']
    method_name = node['methodname']
    desc = node['desc']
    if not fpath in method_db_map:
        method_db_map[fpath] = {}
    if not class_name in method_db_map[fpath]:
        method_db_map[fpath][class_name] = {}
    method_db_map[fpath][class_name][method_name] = desc

In [17]:
def gen_prompt_method(class_desc:str,
                      method_code:str,
                      max_length:int=1024):
    '''
    class_desc: 方法所在的类的描述
    method_code: 代码内容    
    max_length: 最大长度
    '''
    template_pref = f"""你是一个资深的软件工程师，现在需要对一个 csharp 软件项目进行分析，并撰写代码文档，现在针对一个文件中类class中包含的method功能以及其作用进行描述。现在给定当前类的功能描述以及方法代码，现在需要对这个方法进行分析，并撰写方法的描述文档：# 方法的的参数含义以及返回值的含义; # 方法所实现的功能; # 方法是如何实现的，使用了什么算法。
    该方法对应的类的描述如下：{class_desc[:max_length]}
    
    方法代码如下:{method_code[:max_length]}
    """
    
    template_surf = '\n\n请生成对应的命名空间描述文档：\n'
    
    return template_pref + template_pref

In [18]:
db_method = tinydb.TinyDB('../data/results/amc-rough_method_db-full.json')

In [20]:
count = 0
for idx,key in enumerate(project_meta):
    node = project_meta[key]
    # 推断文件编码格式
    fpath = node['filepath']
    allref_lst = node['source_reference'] + list(node['relocal_reference'])
    ref_desc_map = {k:('' if not k in namespace_db_map else namespace_db_map[k]) for k in allref_lst}
    class_map_lst = node['source_class']
    for class_map_node in class_map_lst:
        class_name = class_map_node['class_name']
        class_desc_str = class_db_map[fpath][class_name]
        
        method_map_lst = class_map_node['methods']
        for method_node in method_map_lst:
            count += 1
            method_name = method_node['method_name']
            query = Query()
            if db_method.contains(query.filepath==fpath and query.classname==class_name and query.methodname==method_name):
                print(f'Count {count}: existing {fpath} - {class_name} - {method_name}')
                continue
                
            
            method_comment = method_node['method_comment']
            method_code = method_node['method_code']
            method_prefix = method_node['method_prefix']

            prompt_str = gen_prompt_method(class_desc=class_desc_str,method_code=method_code)
            
            pt_llama = new_p = f"""<|im_start|>使用中文的csharp资深开发者 "\n {prompt_str} 使用中文回答，尽可能的详尽描述每个部分的功能，以及总结整体代码功能。<|im_end|>\n"""
        
            messages=[
                { 'role': 'user', 'content': pt_llama}
            ]
            
            inputs = tokenizer.apply_chat_template(messages, 
                                                   add_generation_prompt=True, 
                                                   return_tensors="pt").to(model.device)
            # tokenizer.eos_token_id is the id of <|EOT|> token
            outputs = model.generate(inputs, max_new_tokens=4086, 
                                     do_sample=False, 
                                     top_k=50, 
                                     top_p=0.95, 
                                     num_return_sequences=1, 
                                     eos_token_id=tokenizer.eos_token_id)
            desc = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
        
            # 插入数据库
            db_method.insert({'filepath': fpath, 'classname':class_name,'methodname':method_name ,'desc': desc})
            print(f'Count {count}: {fpath} - {class_name} - {method_name} insert sucsscfull.')


Count 1: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - FindFirstPostionInFSFGate
Count 2: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - FindFirstPostionRelativeInGate
Count 3: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - CalCouplantDepth
Count 4: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - DataToColor
Count 5: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - DataToColorForInt
Count 6: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - DataToColorForInt
Count 7: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - OutOldColorBar
Count 8: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - AdjustPeakData
Count 9: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFunction - CheckFileFormat
Count 10: existing ../data/AMC/AlgorithmLib/CScanPicFunction.cs - CScanPicFu

/home/lawbda/envs/anaconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Count 408: existing ../data/AMC/AMC/Form1.cs - Form1 - IsSaveCScanPic
Count 409: existing ../data/AMC/AMC/Form1.cs - Form1 - IniReadList
Count 410: existing ../data/AMC/AMC/Form1.cs - Form1 - ChangeButtonAboutFullWave
Count 411: existing ../data/AMC/AMC/Form1.cs - Form1 - AddCheckboxListForFullWave
Count 412: existing ../data/AMC/AMC/Form1.cs - Form1 - GoToWorkPartZeroForOpenFile
Count 413: existing ../data/AMC/AMC/Form1.cs - Form1 - GlobalParameterSet_ItemClick
Count 414: existing ../data/AMC/AMC/Form1.cs - Form1 - ClearGlobalParameterFrm
Count 415: existing ../data/AMC/AMC/Form1.cs - Form1 - SetSecondScanParameters
Count 416: existing ../data/AMC/AMC/Form1.cs - Form1 - IsSaveData
Count 417: existing ../data/AMC/AMC/Form1.cs - Form1 - toolStripSavePic_ItemClick
Count 418: existing ../data/AMC/AMC/Form1.cs - Form1 - toolStripOpenConfig_ItemClick
Count 419: existing ../data/AMC/AMC/Form1.cs - Form1 - LoadParameters
Count 420: existing ../data/AMC/AMC/Form1.cs - Form1 - MotionXYZForParas

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Count 425: ../data/AMC/AMC/Form1.cs - Form1 - SaveConfigGailSpeed insert sucsscfull.


...

Count 3980: ../data/AMC/XmlConfigs/CorrectWave.cs - CorrectWave - InitWaveSetting insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Count 3981: ../data/AMC/XmlConfigs/DPRConfig.cs - DPRConfig - InitDPRSetting insert sucsscfull.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Count 3982: ../data/AMC/XmlConfigs/ScanAxis.cs - ScanAxis - IndexAxisesToDictionary insert sucsscfull.
Count 3983: ../data/AMC/XmlConfigs/ScanSetting.cs - ScanSetting - DefaultEnableAxisToArray insert sucsscfull.
